## Estimating the impact of AuxTel mount oscillations on Image Quality.
Craig Lage 06-May-22

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION

In [ ]:
def dAzdT(ALT, AZ):
    # I tried doing this anlytically, but then went to
    # this brute force method that is guaranteed to work.
    deltaT = 10.0
    time = datetime.now()
    shiftedTime = time + timedelta(seconds=deltaT)
    altAz = AltAz(obstime=time, location=AUXTEL_LOCATION)
    shiftedAltAz = AltAz(obstime=shiftedTime, location=AUXTEL_LOCATION)
    skyLocation = SkyCoord(AZ, ALT, frame=altAz)
    az1 = skyLocation.altaz.az.deg
    shiftedSkyLocation = skyLocation.transform_to(shiftedAltAz)
    az2 = shiftedSkyLocation.altaz.az.deg
    deltaAz = abs(az1 - az2)
    if deltaAz > 1.0:
        deltaAz = abs(deltaAz - 360.0)
    azSpeed = abs(deltaAz) / deltaT * 3600.0
    if azSpeed > 500.0:
        return 0.0
    else:
        return azSpeed

def AverageDeviation(period, amplitude, exptime):
    nTrials = 100
    times = np.linspace(0, exptime, 100)
    deviations = []
    for i in range(nTrials):
        phase = 2.0 * np.pi * np.random.random()
        motion = amplitude * np.cos(2.0 * np.pi * times / period + phase)
        deviation = motion.std()
        deviations.append(deviation)
        #print(i, phase, deviation)
    return np.mean(deviations)
        
def ImageImpact(ALT, AZ): 
    exptime = 30.0 # seconds
    coggingPeriod = 675.0 # arcseconds
    azSpeed = dAzdT(ALT, AZ)
    period = coggingPeriod / azSpeed
    azFreq = azSpeed / coggingPeriod
    amplitude = 12.96 * azFreq # Empirical relation - arcseconds
    #print(azSpeed, period, azFreq, amplitude)
    imageImpact = AverageDeviation(period, amplitude, exptime) * np.cos(ALT)
    return abs(imageImpact)

In [ ]:
nx = 50
ny = 100

alts = np.linspace(5.0, 80.0, nx)
azs = np.linspace(-179.9, 179.9, ny)
alt_1, az_1 = np.meshgrid(alts, azs)
z = np.zeros([ny, nx])
for i, alt in enumerate(alts):
    for j, az in enumerate(azs):
        z[j,i] = dAzdT(Angle(alt * u.deg), Angle(az * u.deg))
        #print(alt, az, z[i,j])
        

In [ ]:
plt.contourf(alt_1, az_1, z)
plt.colorbar()
plt.xlabel("Altitude (degrees)")
plt.ylabel("Azimuth (degrees)")
plt.title("Azimuth tracking speed (arseconds / second)")


In [ ]:
nx = 100
ny = 200
alts = np.linspace(5.0, 85.0, nx)
azs = np.linspace(-179.9, 179.9, ny)
alt_1, az_1 = np.meshgrid(alts, azs)
IIz = np.zeros([ny, nx])
for i, alt in enumerate(alts):
    for j, az in enumerate(azs):
        IIz[j,i] = ImageImpact(Angle(alt * u.deg), Angle(az * u.deg))

plt.contourf(alt_1, az_1, IIz)
plt.colorbar() 
plt.xlabel("Altitude (degrees)")
plt.ylabel("Azimuth (degrees)")
plt.title("Image impact of mount oscillations (arcseconds)")


In [ ]:
plt.contourf(alt_1, az_1, IIz)
plt.colorbar() 
plt.xlabel("Altitude (degrees)")
plt.ylabel("Azimuth (degrees)")
plt.title("Image impact of mount oscillations (arcseconds)")

plt.savefig("/home/craiglagegit/DATA/Oscillation_Image_Impact_06May22.png")